# Pedro - Short Queeze Predictor
---

### Imports

In [58]:
# Import necessary libraries for the project
import os
import numpy as np
import pandas as pd
import yfinance as yf
from glob import glob
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import pandas_market_calendars as mcal
from sklearn.preprocessing import StandardScaler,OneHotEncoder

### Environment Variables

In [32]:
# Load environment variables from a .env file.
load_dotenv('alpaca.env')

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Initialize Alpaca API
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2"
)

### Short Interest Data Collection

In [46]:
# Load and preprocess short interest data
short_df = pd.read_csv(Path("Resources/ShortFloat.csv"))
short_df.rename(columns={'ShortSqueeze.com Short Interest Data': 'Company Name'}, inplace=True)

# Dropping irrelevant columns
columns_to_drop = [
    'Total Short Interest', 'Days to Cover', 'Performance (52-wk)', 'Short: Prior Mo', '% Change Mo/Mo',
    'Shares: Float', 'Avg. Daily Vol.', 'Shares: Outstanding', 'Short Squeeze Ranking™', '% from 52-wk High',
    '(abs)', '% from 200 day MA', '(abs).1', '% from 50 day MA', '(abs).2', '% Insider Ownership',
    '% Institutional Ownership'
]
columns_to_drop = [col for col in columns_to_drop if col in short_df.columns]
short_df.drop(columns_to_drop, axis=1, inplace=True)

# Convert 'Short % of Float' and 'Market Cap' to numeric and apply filters
short_df['Short % of Float'] = pd.to_numeric(short_df['Short % of Float'], errors='coerce')
short_df = short_df[short_df['Short % of Float'] >= 17]
short_df['Market Cap'] = pd.to_numeric(short_df['Market Cap'], errors='coerce')
short_df = short_df[short_df['Market Cap'] >= 300000000]

/Users/regulolanz/anaconda3/envs/dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Date Cleaning and Mapping

In [47]:
# Convert 'Record Date' column to datetime, sort the dataframe by 'Record Date'.
date_mapping = {
    'JanA': '01-11', 'JanB': '01-25',
    'FebA': '02-09', 'FebB': '02-27',
    'MarA': '03-09', 'MarB': '03-24',
    'AprA': '04-12', 'AprB': '04-25',
    'MayA': '05-09', 'MayB': '05-24',
    'JunA': '06-09', 'JunB': '06-27',
    'JulA': '07-12', 'JulB': '07-25',
    'AugA': '08-09', 'AugB': '08-24',
    'SepA': '09-12', 'SepB': '09-26',
    'OctA': '10-10', 'OctB': '10-24',
    'NovA': '11-09', 'NovB': '11-27',
    'DecA': '12-11', 'DecB': '12-27',
}
short_df['Record Date'] = pd.to_datetime(short_df['Record Date'].str.replace(r'(\d{4})-(\w+)', lambda m: f'{m.group(1)}-{date_mapping[m.group(2)]}'))
short_df.sort_values('Record Date', inplace=True)
short_df.reset_index(drop=True, inplace=True)

/Users/regulolanz/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  


### Insider Trading Data Collection

In [48]:
# Load insider trading data, remove dollar symbols, and convert columns to numeric
insider_df = pd.read_csv("Resources/InsiderTrading.csv")
insider_df['Total Amount'] = insider_df['Total Amount'].replace({'\$': '', ',': ''}, regex=True).astype(float)
insider_df['Share Price'] = insider_df['Share Price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
insider_df['Date'] = pd.to_datetime(insider_df['Date'])
insider_df['Total Amount'] = pd.to_numeric(insider_df['Total Amount'], errors='coerce')
insider_df = insider_df[insider_df['Total Amount'] >= 1000000]

### Data Merge

In [49]:
# Merge short_df and insider_df based on the Symbol column, rename columns, and select necessary columns
merged_df = pd.merge(short_df, insider_df, on='Symbol')
merged_df['Share Price'] = merged_df['Share Price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
merged_df.rename(columns={'Share Price': 'Close Price'}, inplace=True)
merged_df = merged_df[['Symbol', 'Short % of Float', 'Total Amount', 'Record Date', 'Close Price', 'Company Name', 'Sector', 'Industry', 'Date']]

### Data Cleaning and Filtering

In [50]:
# Calculate the difference between 'Date' and 'Record Date' for each row
merged_df['Date_diff'] = (merged_df['Date'] - merged_df['Record Date']).dt.days

# Filter out rows where 'Date_diff' is more than 30 and drop unnecessary columns
merged_df = merged_df[merged_df['Date_diff'] >= 0]
merged_df.sort_values(['Symbol', 'Date_diff'], inplace=True)
merged_df.drop_duplicates(subset='Symbol', keep='first', inplace=True)
merged_df = merged_df[merged_df['Date_diff'] <= 30]
merged_df.drop(columns=['Record Date', 'Date_diff'], inplace=True)

# Reorder columns
new_column_order = ['Symbol', 'Short % of Float', 'Total Amount', 'Date', 'Close Price', 'Company Name', 'Sector', 'Industry']
merged_df = merged_df[new_column_order]

### Data Augmentation

In [51]:
# Create new columns for Close Prices at future dates and calculate Returns.
nyse = mcal.get_calendar('NYSE')
desired_days = [2, 3, 4, 5, 7]
for day in desired_days:
    merged_df[f'Close Price Day {day}'] = np.nan

for idx, row in merged_df.iterrows():
    trading_days = nyse.valid_days(start_date=row['Date'], end_date=row['Date'] + pd.DateOffset(days=10))

    for day in desired_days:
        if day <= len(trading_days):
            data = yf.download(row['Symbol'], start=trading_days[day - 1], end=trading_days[day - 1] + pd.DateOffset(days=1))
            if not data.empty:  
                merged_df.loc[idx, f'Close Price Day {day}'] = data['Close'][0] 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

### Calculate Returns and Highest Day Return

In [52]:
for day in [5, 7]: 
    merged_df[f'Return ({day} Days)'] = ((merged_df[f'Close Price Day {day}'] - merged_df['Close Price']) / merged_df['Close Price']) * 100

merged_df['Highest Day Return'] = merged_df[[f'Return ({day} Days)' for day in [5, 7]]].max(axis=1)
merged_df['Highest Close Price'] = merged_df[[f'Close Price Day {day}' for day in desired_days]].max(axis=1)

# Format the 'Close Price' and 'Return' columns to have 2 decimal places
for col in merged_df.columns:
    if 'Close Price' in col or 'Return' in col:
        merged_df[col] = merged_df[col].round(2)

# Drop NaNs and reset the index
merged_df.dropna(inplace=True)
merged_df.reset_index(drop=True, inplace=True)

In [53]:
merged_df

,Symbol,Short % of Float,Total Amount,Date,Close Price,Company Name,Sector,Industry,Close Price Day 2,Close Price Day 3,Close Price Day 4,Close Price Day 5,Close Price Day 7,Return (5 Days),Return (7 Days),Highest Day Return,Highest Close Price
0,DISH,20.22,6210000.00,2023-05-11,6.21,Dish Network Corporation - Class A,Communication Services,Entertainment,6.16,6.57,6.48,6.86,6.70,10.47,7.89,10.47,6.86
1,EVA,19.90,1022000.00,2023-05-08,10.22,Enviva Partners Lpunits Representing Limi,Basic Materials,Lumber & Wood Production,9.02,9.46,8.69,8.44,8.01,-17.42,-21.62,-17.42,9.46
2,FATE,29.95,1586029.02,2023-04-21,6.18,Fate Therapeutics Inc,Healthcare,Biotechnology,6.34,6.03,5.82,5.84,6.36,-5.50,2.91,2.91,6.36
3,KSS,19.97,2018350.00,2023-03-29,21.82,Kohls Corporation,Consumer Cyclical,Department Stores,22.88,23.54,23.90,24.01,22.66,10.04,3.85,10.04,24.01
4,LAZR,29.73,7574970.00,2023-05-15,5.83,Luminar Technologies Inc. - Class A Common Stock,Technology,Software - Application,6.01,6.54,6.47,6.27,6.46,7.55,10.81,10.81,6.54
5,RCUS,21.54,19452600.00,2023-06-28,19.26,Arcus Biosciences Inc,Healthcare,Biotechnology,19.71,20.31,20.44,20.67,20.37,7.32,5.76,7.32,20.67
6,RILY,19.66,5460788.08,2023-02-08,40.18,B. Riley Financial Inc,Financial Services,Financial Conglomerates,39.53,40.78,43.76,43.49,43.91,8.24,9.28,9.28,43.91
7,SAVA,27.62,1990629.76,2023-03-07,25.76,Cassava Sciences Inc,Healthcare,Biotechnology,24.97,24.79,24.90,25.62,26.05,-0.54,1.13,1.13,26.05


In [54]:
# Transpose the dataframe for further analysis
transposed_df = merged_df.T
transposed_df = transposed_df.drop(['Company Name', 'Date', 'Sector', 'Industry'])
transposed_df.set_axis(transposed_df.loc['Symbol'], axis=1, inplace=True)
transposed_df.drop('Symbol', inplace=True)

# Display the transposed dataframe
transposed_df

Symbol,DISH,EVA,FATE,KSS,LAZR,RCUS,RILY,SAVA
Short % of Float,20.22,19.9,29.95,19.97,29.73,21.54,19.66,27.62
Total Amount,6210000.0,1022000.0,1586029.02,2018350.0,7574970.0,19452600.0,5460788.08,1990629.76
Close Price,6.21,10.22,6.18,21.82,5.83,19.26,40.18,25.76
Close Price Day 2,6.16,9.02,6.34,22.88,6.01,19.71,39.53,24.97
Close Price Day 3,6.57,9.46,6.03,23.54,6.54,20.31,40.78,24.79
Close Price Day 4,6.48,8.69,5.82,23.9,6.47,20.44,43.76,24.9
Close Price Day 5,6.86,8.44,5.84,24.01,6.27,20.67,43.49,25.62
Close Price Day 7,6.7,8.01,6.36,22.66,6.46,20.37,43.91,26.05
Return (5 Days),10.47,-17.42,-5.5,10.04,7.55,7.32,8.24,-0.54
Return (7 Days),7.89,-21.62,2.91,3.85,10.81,5.76,9.28,1.13


In [55]:
# Review the data types associated with the columns
merged_df.dtypes

Symbol                         object
Short % of Float              float64
Total Amount                  float64
Date                   datetime64[ns]
Close Price                   float64
Company Name                   object
Sector                         object
Industry                       object
Close Price Day 2             float64
Close Price Day 3             float64
Close Price Day 4             float64
Close Price Day 5             float64
Close Price Day 7             float64
Return (5 Days)               float64
Return (7 Days)               float64
Highest Day Return            float64
Highest Close Price           float64
dtype: object

In [56]:
# Create a list of categorical variables 
categorical_variables = list(merged_df.select_dtypes("object").columns)

# Display the categorical variables list
display(categorical_variables)

['Symbol', 'Company Name', 'Sector', 'Industry']

In [59]:
enc = OneHotEncoder(sparse=False)

In [60]:
# Encode the categorical variables using OneHotEncoder
encoded_data = enc.fit_transform(merged_df[categorical_variables])

In [61]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(encoded_data, columns = enc.get_feature_names(categorical_variables))

# Review the DataFrame
encoded_df.head()

/Users/regulolanz/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,Symbol_DISH,Symbol_EVA,Symbol_FATE,Symbol_KSS,Symbol_LAZR,Symbol_RCUS,Symbol_RILY,Symbol_SAVA,Company Name_Arcus Biosciences Inc,Company Name_B. Riley Financial Inc,...,Sector_Consumer Cyclical,Sector_Financial Services,Sector_Healthcare,Sector_Technology,Industry_Biotechnology,Industry_Department Stores,Industry_Entertainment,Industry_Financial Conglomerates,Industry_Lumber & Wood Production,Industry_Software - Application
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [62]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat([merged_df.select_dtypes(["int64", "float64"]), encoded_df], axis=1)

# Review the DataFrame
encoded_df.head()

,Short % of Float,Total Amount,Close Price,Close Price Day 2,Close Price Day 3,Close Price Day 4,Close Price Day 5,Close Price Day 7,Return (5 Days),Return (7 Days),...,Sector_Consumer Cyclical,Sector_Financial Services,Sector_Healthcare,Sector_Technology,Industry_Biotechnology,Industry_Department Stores,Industry_Entertainment,Industry_Financial Conglomerates,Industry_Lumber & Wood Production,Industry_Software - Application
0,20.22,6210000.00,6.21,6.16,6.57,6.48,6.86,6.70,10.47,7.89,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,19.90,1022000.00,10.22,9.02,9.46,8.69,8.44,8.01,-17.42,-21.62,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,29.95,1586029.02,6.18,6.34,6.03,5.82,5.84,6.36,-5.50,2.91,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,19.97,2018350.00,21.82,22.88,23.54,23.90,24.01,22.66,10.04,3.85,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,29.73,7574970.00,5.83,6.01,6.54,6.47,6.27,6.46,7.55,10.81,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
